# Bounding Box
When you run the [ObjectDetection Workflow](https://docs.aperturedata.io/workflows/object_detection), bounding boxes are computed for each image stored in ApertureDB. You can use this notebook to query and visualize the results. For comprehensive details on bounding boxes, polygons, and other region-based features in ApertureDB, please refer to [ApertureDB Image documentation](https://docs.aperturedata.io/category/image-commands). For details on the Image interface in the ApertureDB python SDK, please refer to the [python SDK documentation](https://docs.aperturedata.io/python_sdk/object_wrappers/Images).

In [ ]:
from aperturedb.CommonLibrary import create_connector, execute_query

client = create_connector(key="<YOUR API KEY HERE>")
response, _ = client.query([{"GetStatus": {}}])
print(response)

## Display BoundingBoxes using the Python object mapper API

To overlay bounding boxes on the image, pass show_bboxes=True as an argument for display function. You can also add constraints to filter the results based on your desired condition.

In [ ]:
from aperturedb.Images import Images
from aperturedb.Constraints import Constraints

display_limit = 5
imgs  = Images(client, batch_size=display_limit)
const = Constraints()
# const.equal("constraint param", "constraint value")
imgs.search(const)
imgs.display(limit=display_limit, show_bboxes=True)

## Find Images using Labels

In [ ]:
query = [{
        "FindBoundingBox": {
            "_ref": 1,
            "with_label": "chair",
            "labels": True,
            "coordinates": True
        }
    },
    {
        "FindImage": {
            "blobs": True,
            "is_connected_to": {
                "ref": 1
            },
            "results": {
                "limit": display_limit,
                "all_properties": True
            }
        }      
    }]

result, response, blobs = execute_query(client, query)
# print(response[1])
if result == 0:
    imgs = Images(client, batch_size= display_limit, response=response[1]["FindImage"]["entities"])
    imgs.display( show_bboxes= True)

## Crop Bounding Boxes

In [ ]:
from aperturedb.NotebookHelpers import display as display_images

query = [{
    "FindBoundingBox": {
        "blobs": True,
        "with_label": "person",
        "results": {
            "all_properties": True,
        },
        "limit": 5
    }}]

result, response, blobs = execute_query(client, query)
# print(response)
if result == 0:
    # display(len(blobs))
    display_images(blobs)